# Install Libraries

In [1]:
!pip install -r /kaggle/input/requirements-file/requirements.txt

In [2]:
!pip install -U -q "bitsandbytes>=0.41.0" "transformers>=4.36.0" "accelerate>=0.26.0"

# Import Libraries

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

import torch
import pandas as pd
import os

2025-12-02 22:11:45.964609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764713505.986541     228 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764713505.993204     228 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Load Data

In [45]:
try:
    df = pd.read_csv('/kaggle/input/tmdb-movies-processed/processed_movies.csv')
    df['page_content'] = df['page_content'].fillna("")
    print(f"Successfully loaded {len(df)} movies.")
except FileNotFoundError:
    print("Error: Could not find 'processed_movies.csv'.")

Successfully loaded 4803 movies.


# Load Embedding Model

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embedding Model Loaded")

Embedding Model Loaded


# Prepare Document

In [6]:
documents = []

In [47]:
for i, r in df.iterrows():
    doc = Document(
        page_content=r['page_content'],
        metadata={
            'id': r['id'],
            'title': r['title'],
            'rating': r['vote_average'],
            'year': r['year'],
            'genres': r['genre_names']
        }
    )
    documents.append(doc)

# Create Database

In [48]:
print(f'Ingesting {len(documents)} movies into ChromaDB...')

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory='./chroma_db_data',
)

print("✅ Success! Database created.")

Ingesting 9606 movies into ChromaDB...
✅ Success! Database created.


# Test

In [9]:
query = 'A movie about a psychological mind game'
print(f"\nTest Query: '{query}'")

results = vector_db.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\nResult {i+1}: {doc.metadata['title']} (Rating: {doc.metadata['rating']})")
    snippest = doc.page_content.replace('\n', ' ').strip()[:150]
    print(f'Context: {snippest}...')


Test Query: 'A movie about a psychological mind game'

Result 1: A Beautiful Mind (Rating: 7.7)
Context: Movie Title: A Beautiful Mind     Director: Ron Howard     Cast: Russell Crowe, Ed Harris, Jennifer Connelly, Christopher Plummer, Paul Bettany, Adam ...

Result 2: A Beautiful Mind (Rating: 7.7)
Context: Movie Title: A Beautiful Mind     Director: Ron Howard     Cast: Russell Crowe, Ed Harris, Jennifer Connelly, Christopher Plummer, Paul Bettany, Adam ...

Result 3: Spider (Rating: 6.4)
Context: Movie Title: Spider     Director: David Cronenberg     Cast: Ralph Fiennes, Miranda Richardson, Gabriel Byrne, Lynn Redgrave, John Neville, Philip Cra...


# Chat Bot

## Setup Model

In [10]:
model_id = "HuggingFaceH4/zephyr-7b-beta"

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [13]:
# Create the pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=512,
)

Device set to use cuda:0


In [17]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## Prompt

In [37]:
prompt_template = """
<|system|>
You are a movie database assistant. 
You are FORBIDDEN from using your own knowledge. 
You must answer ONLY using the Context provided below.
If the Context is empty or does not contain the answer, you must say: "I have no movies in my database that match your request."
</s>
<|user|>
Context:
{context}

Question: 
{question}
</s>
<|assistant|>
"""

In [38]:
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question']
)

## Build Chain

In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',  # to stuff all results into one prompt
    # "fetch_k": 10 means "look at 10 movies first"
    # "k": 4 means "only return the top 4 distinct ones"
    # "lambda_mult": 0.25 ensures diversity so you don't get duplicates
    retriever=vector_db.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 4, 'lambda_mult': 0.25},
                                    ),
    chain_type_kwargs={'prompt': PROMPT},
)

## Run Bot

In [27]:
print('Bot is ready! Ask away.')
user_input = 'I want a movie where the bad guy wins'
response = qa_chain.invoke(user_input)

print('\n--------------------------------------------------------')
print(f'User: {user_input}')
print(f"Bot: {response['result']}")
print('\n--------------------------------------------------------')

Bot is ready! Ask away.

--------------------------------------------------------
User: I want a movie where the bad guy wins
Bot: Based on your request, none of the movies listed above fit your criteria. In all of them, the good guys ultimately prevail over the bad guys. However, if you're looking for a movie where the bad guy wins, I would suggest checking out "No Country for Old Men" (directed by Joel Coen and Ethan Coen) or "The Departed" (directed by Martin Scorsese). Both of these films have complex plots and unexpected twists that lead to surprising endings. In "No Country for Old Men," the main antagonist, played by Javier Bardem, seems almost invincible at times, making his eventual victory all the more shocking. And in "The Departed," the line between good and evil becomes increasingly blurred as the two main characters, played by Leonardo DiCaprio and Matt Damon, struggle to outsmart each other. While both movies have their fair share of heroes, the villains are just as comp

In [35]:
# Test the bot with searching in the document
print("Searching database for evidence...")
docs = vector_db.similarity_search("I want action movie after 2015", k=4)

print("\n--- WHAT THE DATABASE FOUND ---")
for i, doc in enumerate(docs):
    print(f"\n[Document {i+1}]: {doc.metadata['title']}")
    print(f"Snippet: {doc.page_content.replace(chr(10), ' ')[:200]}...") # Clean up newlines

Searching database for evidence...

--- WHAT THE DATABASE FOUND ---

[Document 1]: eXistenZ
Snippet:      Movie Title: eXistenZ     Director: David Cronenberg     Cast: Jennifer Jason Leigh, Jude Law, Ian Holm, Willem Dafoe, Don McKellar, Oscar Hsu, Callum Keith Rennie, Christopher Eccleston, Sarah P...

[Document 2]: eXistenZ
Snippet:      Movie Title: eXistenZ     Director: David Cronenberg     Cast: Jennifer Jason Leigh, Jude Law, Ian Holm, Willem Dafoe, Don McKellar, Oscar Hsu, Callum Keith Rennie, Christopher Eccleston, Sarah P...

[Document 3]: Last Action Hero
Snippet:      Movie Title: Last Action Hero     Director: John McTiernan     Cast: Arnold Schwarzenegger, F. Murray Abraham, Art Carney, Charles Dance, Frank McRae, Tom Noonan, Anthony Quinn, Mercedes Ruehl, A...

[Document 4]: Last Action Hero
Snippet:      Movie Title: Last Action Hero     Director: John McTiernan     Cast: Arnold Schwarzenegger, F. Murray Abraham, Art Carney, Charles Dance, Frank McRae, Tom Noonan, Anth

In [29]:
print("Testing with a solvable query...")
query = "I want a dystopian movie where teenagers fight to the death"
response = qa_chain.invoke(query)

print(f"\nUser: {query}")
print(f"Bot:  {response['result']}")

Testing with a solvable query...

User: I want a dystopian movie where teenagers fight to the death
Bot:  Based on the context provided, "The Hunger Games" (directed by Gary Ross) might be what you're looking for. It's a dystopian movie where teenagers fight to the death, based on Suzanne Collins' bestselling novel. The main character, Katniss Everdeen (played by Jennifer Lawrence), volunteers to participate in the annual Hunger Games, a televised event in which one boy and one girl from each district are selected to fight to the death until only one survivor remains. The movie explores themes of survival, rebellion, and social inequality in a futuristic society. Other cast members include Josh Hutcherson, Liam Hemsworth, Woody Harrelson, Elizabeth Banks, Lenny Kravitz, Stanley Tucci, and Donald Sutherland. The movie received critical acclaim and was a commercial success, grossing over $694 million worldwide.


In [39]:
print("Testing with a solvable query...")
query = "I want action movie after 2020"
response = qa_chain.invoke(query)

print(f"\nUser: {query}")
print(f"Bot:  {response['result']}")

Testing with a solvable query...

User: I want action movie after 2020
Bot:  Based on the information provided, some action movies released after 2020 that you might enjoy include:

1. Tenet (2020) - Directed by Christopher Nolan, starring John David Washington, Robert Pattinson, Elizabeth Debicki, and Kenneth Branagh. Genres: Action, Sci-Fi, Thriller.

2. Mulan (2020) - Live-action adaptation of the Disney animated classic, directed by Niki Caro, starring Yifei Li, Donnie Yen, Jason Scott Lee, and Gong Li. Genres: Action, Adventure, Drama, Family, Fantasy.

3. Greenland (2020) - Disaster movie about a family trying to escape a catastrophic event, directed by Ricardo de Montreuil, starring Gerard Butler, Morena Baccarin, and Roger Dale Floyd. Genres: Action, Drama, Sci-Fi, Thriller.

4. The Old Guard (2020) - Adaptation of the comic book series, directed by Gina Prince-Bythewood, starring Charlize Theron, KiKi Layne, Matthias Schoenaerts, and Chiwetel Ejiofor. Genres: Action, Fantasy, 

In [53]:
print("--- TRUTH CHECK ---")
query = "I want action movie after 2016"

# Get the retriever using the function above
smart_retriever = get_smart_retriever(query, vector_db)

# Ask the database directly
docs = smart_retriever.invoke(query)

if len(docs) == 0:
    print("\n❌ The database returned ZERO movies.")
    print("Conclusion: The dataset is too old. The bot used its own memory (hallucination) to answer.")
else:
    for doc in docs:
        print(f"Found: {doc.metadata['title']} ({doc.metadata['year']})")

--- TRUTH CHECK ---
🔎 Detected Filter: Year > 2016
Found: Growing Up Smith (2017)
